# Machine Learning on graphs
Here we review some simple yet powerful machine learning on graphs

In [ ]:
import numpy as np
import networkx as nx
import community as community_louvain
import matplotlib.pyplot as plt

# Clustering and community detection (unsupervised learning)
Many real-world networks posesses communities, groups of nodes that are more connected together than with the rest of the network. Detecting these structures is of high importance. It reveals the hierarchy, the organization and interactions between nodes. It helps classifying parts of a network into categories. It can be seen as an equivalent of the unsupervised learning `k-means` clustering algorithm. Instead of computing distances and grouping data points in a high dimensional space, we use the network structure to detect the clusters.

Many methods exists for community detection, see for example [this review](https://arxiv.org/abs/0906.0612). We will see two of them. The first one is available in `networkx`, the second one is the popular Louvain method which allows for fast computation.

## Girvan Newman algorithm
The [GN algorithm](https://en.wikipedia.org/wiki/Girvan%E2%80%93Newman_algorithm) remove edges between communities in an iterative manner. The edges removed are the ones with the highest number of shortest paths passing through them ("bottlenecks" between communities). The idea is clear and intuitive. However, the computation is intensive as all shortest paths have to be computed. Moreover, the number of communities has to be specified as this algorithm do not have a stopping criterium (this can be an advantage or drawback).

In [ ]:
# Girvan Newman clustering
G = nx.path_graph(8)

num_clusters = 5 # desired number of clusters
k = num_clusters - 1

comp = nx.algorithms.community.centrality.girvan_newman(G)

comp = list(comp)
for idx in range(k):
    print(' {} communities: {}'.format(idx+2, comp[idx]))

# Alternative way using directly the generator
#import itertools
#for idx,communities in enumerate(itertools.islice(comp, k)):
#    print(' {} communities: {}'.format(idx+2, tuple(sorted(c) for c in communities)))

## Exercise
* Apply it with a more complex network and visualize the communities using Gephi. (See [graph list in networkx](https://networkx.org/documentation/stable/reference/generators.html), you may test on the "Karate club" graph or "Les Miserables" graph)
* Try a larger network and experience the limit of scalability. What is a reasonable number of nodes for this method?

## Louvain community detection

Community detection with Louvain method. You have to install an external module: `pip install louvain`, see [module Github page](https://github.com/taynaud/python-louvain) for more info or the [paper](https://arxiv.org/abs/0803.0476). This method is much more efficient than the previous one. It is a greedy, non-parametric, algorithm that finds automatically the optimal number of communities.

In [ ]:
#Louvain module is called "community"
partition = community_louvain.best_partition(G)
#community_louvain.modularity(partition, G)

`partition` is a dictionary where each node id is a key and its community is the value.

In [ ]:
# re-order partition to have a dictionary of clsuters
clusters = {}
for i, v in partition.items():
    clusters[v] = [i] if v not in clusters.keys() else clusters[v] + [i]
print(clusters)

## Label propagation (Semi-supervised learning)
In this approach, the graph structure is combined to values (or feature vectors) associated to the nodes. Missing node values are found by propagating the known values to their neighbors.

In [ ]:
L = nx.normalized_laplacian_matrix(G)

We use label spreading from this [publication](http://citeseer.ist.psu.edu/viewdoc/summary?doi=10.1.1.115.3219) which perform a smoothing of the labels over the graph. We assume 2 classes with one-hot-encoding,i.e. feature vectors on nodes have dimension 2.

In [ ]:
# labels
labels = np.zeros((L.shape[0],2))
labels[1,0] = 1 # node 1 has first label  
labels[5,1] = 1 # node 5 has second label
print(labels)

In [ ]:
def labelSpreading(G, labels, alpha, tol=1e-3):
    L = nx.normalized_laplacian_matrix(G)
    S = np.identity(L.shape[0]) - L.toarray()
    max_iter = 1000
    Y = np.zeros(labels.shape)   
    for i in range(max_iter):
        Y_tmp = Y.copy()
        Y = alpha * np.dot(S, Y) + (1 - alpha) * labels
        if np.linalg.norm(Y-Y_tmp) < tol:
            print('Converged after {} iterations.'.format(i))
            break
    return Y

In [ ]:
smooth = labelSpreading(G,labels, 0.9)
print(labels)
print(smooth)

Let us plot the results

In [ ]:
pos = nx.spring_layout(G, iterations=200)

plt.figure(figsize=(14, 6))
ax1 = plt.subplot(1, 2, 1)
ax2 = plt.subplot(1, 2, 2)
#ax3 = plt.subplot(1, 4, 3)
nx.draw(G, pos=pos, ax=ax1, node_color=smooth[:,0],  cmap=plt.cm.Blues)
nx.draw(G, pos=pos, ax=ax2, node_color=smooth[:,1],  cmap=plt.cm.Blues)

ax1.title.set_text("Smoothing of class 1 over the network")
ax2.title.set_text("Smoothing of class 2 over the network")
plt.show()

In [ ]:
propagated_labels = np.argmax(smooth,axis=1)
#label_dic = {k:v for k,v in enumerate(propagated_labels)}
print(propagated_labels)

In [ ]:
nx.draw(G, pos=pos, node_color=propagated_labels,  cmap=plt.cm.Blues)
plt.title("Final classification")
plt.show()